In [19]:
!pip install tensorflow opencv-python mediapipe sklearn matplotlib gtts playsound==1.2.2

In [20]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time 
import gtts 
import playsound
import mediapipe as mp



In [21]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils


In [22]:
def mediapipe_detect(img , model):
    img = cv2.cvtColor(img , cv2.COLOR_BGR2RGB)
    img.flags.writeable = False
    results = model.process(img)
    img.flags.writeable = True
    img = cv2.cvtColor(img , cv2.COLOR_RGB2BGR)
    return img , results

In [23]:
def draw_new_landmarks(img , model):
    mp_drawing.draw_landmarks(img , results.pose_landmarks ,mp_holistic.POSE_CONNECTIONS  , mp_drawing.DrawingSpec(color=(80,22,10) , thickness=2 , circle_radius=4) ,mp_drawing.DrawingSpec(color=(80,44,121) , thickness=2 , circle_radius=2))
    mp_drawing.draw_landmarks(img , results.left_hand_landmarks ,mp_holistic.HAND_CONNECTIONS  , mp_drawing.DrawingSpec(color=(121,22,76) , thickness=2 , circle_radius=4) ,mp_drawing.DrawingSpec(color=(121,44,250) , thickness=2 , circle_radius=2))
    mp_drawing.draw_landmarks(img , results.right_hand_landmarks ,mp_holistic.HAND_CONNECTIONS  , mp_drawing.DrawingSpec(color=(245,117,66) , thickness=2 , circle_radius=4) ,mp_drawing.DrawingSpec(color=(245,66,230) , thickness=2 , circle_radius=2))
    

In [24]:
def extract_keypoints(results):
    pose = np.array([[res.x , res.y , res.z , res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    lh = np.array([[res.x , res.y , res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x , res.y , res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose,lh,rh])

In [25]:
DATA_PATH = os.path.join('Data_New')

actions = np.array(['what is your name'])
no_sequences=60
sequence_length=30

In [26]:
for action in actions:
    for sequence in range(no_sequences):
        try:
            os.makedirs(os.path.join(DATA_PATH , action , str(sequence)))
        except:
            pass

In [ ]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
   
      for action in actions:
        for sequence in range(no_sequences):
            for frame_num in range(sequence_length):

                ret, frame = cap.read()

                image, results = mediapipe_detect(frame, holistic)
                draw_new_landmarks(image, results)
                
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                  
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(2000)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                   
                    cv2.imshow('OpenCV Feed', image)
                
              
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
cap.release()
cv2.destroyAllWindows()